# 1. 데이터 준비

In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine-date')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [2]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)
print(train_data.shape, test_data.shape)

(5197, 3) (1300, 3)


# 2. 검증 세트 준비
- 앞서 나눈 훈련 세트와 테스트 세트에서 훈련 세트를 다시 한 번 더 나누어 검증 세트를 준비한다.

In [3]:
sub_data, val_data, sub_target, val_target = train_test_split(train_data, train_target, test_size = 0.2, random_state = 42)
print(sub_data.shape, val_data.shape)


(4157, 3) (1040, 3)


# 3. 훈련/검증 세트 이용해 모델 학습 (Decision Tree Classification)
- 훈련 세트와 검증 세트를 이용해 모델을 학습한다.
- 과대/과소적합이 발생하면 모델의 매개변수를 조절해 과적합을 줄인다.

In [4]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_data, sub_target)
print(dt.score(sub_data, sub_target))
print(dt.score(val_data, val_target)) # overfitting 

0.9971133028626413
0.864423076923077


# 4. 교차 검증 (Cross Validation)
- 검증 세트를 적게 떼어두면 검증 점수가 들쭉날쭉하고 불안정할텐데 이럴 때 교차 검증을 이용하면 안정적인 검증 점수를 얻을 수 있고, 더 많은 데이터를 사용할 수 있다. 좋은 모델을 만들려면 데이터가 많을수록 좋다.
- 검증 세트를 떼어 내어 평가하는 과정을 *여러 번* 반복하는 것이 *교차 검증*
<br/><br/>
- **K-fold Cross Validation**: 교차 검증을 k 번 수행하고 검증 점수의 평균을 내는 것.
- `cross_validate(모델 객체, 훈련 세트 (X, y) 전체)`
    - 기본적으로 5 폴드 교차 검증 수행 (매개변수 `cv` 조정해서 폴드 수 설정 가능)
    - fit time, score time, test score 반환
- `cross_val_score(모델 객체, 훈련 세트 전체)`: test_score 만 반환

- **`cross_val_score()`, `cross_validate()` 함수를 이용해 각 검증 세트의 점수를 확인할 수 있다.**

In [5]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_data, train_target)
print(scores)

{'fit_time': array([0.00487208, 0.00489283, 0.00527096, 0.00504303, 0.00461268]), 'score_time': array([0.00045681, 0.0005672 , 0.00070715, 0.00042534, 0.00038719]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [6]:
    # test_score(검증 세트 정확도 점수)만 반환해주는 cross_val_score 함수
# from sklearn.model_selection import cross_val_score

# scores = cross_val_score(dt, train_data, train_target)
# print(scores)

- **교차 검증의 최종 점수는 test score를 합산해 평균하여 얻을 수 있다.**

In [7]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


## 4-1. 훈련 세트를 섞고 교차 검증 (Splitter)
- 앞선 교차 검증 단계에서는 `train_test_split()` 함수를 이용해 전체 데이터를 섞은 후 훈련 세트를 준비했기 때문에 훈련 세트를 따로 섞을 필요가 없다.
- 교차 검증을 할 때에는 *분할기(Splitter)* 를 지정해 훈련 세트를 섞어줘야 한다.
<br/><br/>
**Splitter**<br>
- 교차 검증에서 폴드를 어떻게 나눌지 결정
- `cross_validate()` 함수: 회귀 모델의 경우 `KFold` 사용, 분류 모델의 경우 `StratifiedKFold` 사용

In [8]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_data, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


- **`cross_validate()` 함수에서 `cv` 매개변수를 조정해 폴드 수를 지정할 수 있다고 했다. 분할기를 사용하는 경우에도 `cv` 매개변수를 이용한다. 폴드 수도 지정하고, 분할기도 쓰려고 할 때에는 분할기 객체 정의 해주고 `n_splits` 매개변수로 폴드 수를 지정할 수 있다.**

In [9]:
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_data, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


# 5. Hyper Parameter Tuning
- 사용자 지정 파라미터: 클래스나 메서드의 매개변수로 표현
- 하이퍼 파라미터 튜닝 자동 수행 기술 'AutoML'

## 5-1. GridSearchCV()
- 여러 매개변수에 대한 최적의 값을 자동으로 찾아주는 클래스
- 자동으로 교차 검증을 해주기 때문에 `cross_validate()` 함수를 호출할 필요가 없다.
```parameter = {'hyper parameters': [values-list]}``` <br>
```GridSearchCV(model, parameter, n_jobs = 1, cv(default = 5))```

In [10]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs = -1)

gs.fit(train_data, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

- 그리드 서치 훈련 후 가장 높은 검증 점수를 가진 모델의 매개 변수 조합이 훈련 세트로 학습된 후 `GridSearchCv().best_estimator_` 속성에 저장됨

In [11]:
dt = gs.best_estimator_
print(dt.score(train_data, train_target))

0.9615162593804117


- 그리드 서치로 찾은 최적의 매개 변수 값은 `GridSearchCv().best_params_` 속성에 저장됨

In [12]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


- 각 매개 변수에서 수행한 교차 검증의 평균 점수 값 `cv_results_['mean_test_score']`에 저장됨

In [13]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


- `cv_results['mean_test_score']`를 확인하지 않고 `np.argmax()`함수를 사용해 가장 큰 값의 인덱스를 추출할 수 있다. 인덱스를 이용해 최상의 값을 만든 파라미터를 확인하거나, 최상의 점수를 확인할 수 있다.

In [14]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])
print(gs.cv_results_['mean_test_score'][best_index])
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}
0.8681929740134745
{'min_impurity_decrease': 0.0001}


## 5-2. GridSearchCV 심화학습

In [15]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }
# np.arange(1, 100, 10): 1부터 100까지 10 단위로 증가하면서 10개의 값을 만들겠다 

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_data, train_target)
dt = gs.best_estimator_
print('best scores:', dt.score(test_data, test_target))
print('best parameters:', gs.best_params_)


best scores: 0.8615384615384616
best parameters: {'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


## 5-3. Random Search (랜덤서치)
- 매개 변수 값이 수치일 때 값의 범위나 간격을 미리 정하기 어렵기 때문에 샘플링할 수 있는 확률 분포 객체를 전달한다.
- `uniform`, `randint`를 이용해 '균등 분포에서 샘플링을 한다'.

- 그리드 서치는 말그대로 모든 경우를 테이블로 만든뒤 격자로 탐색하는 방식
- 랜덤 서치는 하이퍼 파라미터 값을 *랜덤하게* 넣어보고 그중 우수한 값을 보인 하이퍼 파라미터를 활용해 모델을 생성한다는 것이다.

- 랜덤 서치가 더 성능이 좋다.
>* 불필요한 탐색 횟수를 줄인다.
>* 그리드 서치는 우리가 딕셔너리에 지정한 모든 값을 다 탐색해야만 한다. 이는 사람이 임의로 정한 값이기 때문에, 어떤 값이 효과적일지는 알 수 없고, 쨌든 입력은 되었으니 한번씩 모델은 다 생성되어야 하는 상황
>* 랜덤 서치는 위와 같은 상황을 방지한다. 랜덤하게 숫자를 넣은 뒤, 정해진 간격(grid) 사이에 위치한 값들에 대해서도 확률적으로 탐색이 가능하므로, 최적 hyperparameter 값을 더 빨리 찾을 수 있다. 


##### randint (정수값을 뽑는 확률 분포)

In [16]:
from scipy.stats import uniform, randint
rgen = randint(0, 10) # 0 부터 10사이의 범위를 갖는 randint 객체를 만들고
rgen.rvs(10) # 10개의 숫자를 샘플링한다.

array([2, 8, 1, 6, 4, 9, 9, 7, 0, 1])

In [17]:
np.unique(rgen.rvs(1000), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([100, 101, 104, 106,  98,  88, 106, 101, 103,  93]))

##### uniform (실수값을 뽑는 확률 분포)

In [18]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.38823308, 0.06157385, 0.03735566, 0.99710438, 0.45594965,
       0.61030704, 0.61133746, 0.69980265, 0.08643009, 0.84916067])

## 5-4. Randomized Search 실습
- **랜덤서치에 randint와 uniform 클래스 객체를 넘겨주고 총 몇 번을 샘플링해서 최적의 매개변수를 찾으라고 명령할 수 있다. 샘플링 횟수는 시스템 자원이 허락하는 범위 내에서 최대한 크게 하는 것이 좋다.**
- **`RandomizedSearchCV`에서는 `n_iter` 매개변수를 조절해 샘플링 횟수를 지정한다.**

In [19]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001), # 불순도
         'max_depth': randint(20, 50), #가지치기 횟수
         'min_samples_split': randint(2, 25), 
         'min_samples_leaf': randint(1, 25), # 리프 노드가 되기 위한 최소 샘플의 갯수. 자식 노드의 샘플 수가 이 값보다 작을 경우 분할 x
         }

In [20]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params
                        , n_iter = 100, n_jobs = -1, random_state = 42)
    # RandomizedSearchCV 객체 정의: 결정 트리 모델, 매개변수 딕셔너리, 샘플링 100회 반복, 모든 CPU 사용

gs.fit(train_data, train_target) # RandomizedSearchCV 학습
print(gs.best_params_) # 최적의 모델이 갖는 최적의 매개변수 값

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [21]:
dt = gs.best_estimator_ # RandomizedSearchCV 클래스를 이용해 얻은 최적의 매개변수 값을 갖춘 최적의 모델
print(dt.score(train_data, train_target)) # 모델 정확도 훈련 세트로 확인

0.8928227823744468


In [22]:
print(np.max(gs.cv_results_['mean_test_score'])) # 검증 세트의 모델 정확도 중 가장 높은 점수 확인
print(dt.score(test_data, test_target)) # 실제 테스트 세트의 모델 정확도 점수 확인

0.8695428296438884
0.86


### Hyper Parameter Tuning 정리
1. GridSearchCV()
- 여러 매개변수에 대한 최적의 값을 자동으로 찾아준다.
- 매개변수 수치 값에 대한 범위와 간격 지정 (`np.arange(a, b, c)`, `arange(a, b, c)` 이용)
- 자동 교차 검증 (`cross_validate()` 함수 호출 불필요)
<br/><br/>
2. RandomizedSearchCV()
- 여러 매개변수에 대한 최적의 값을 자동으로 찾아준다.
- 수치 값의 범위를 지정 불필요 (`uniform(a, b)`,`randint(a, b)` 이용)
- 자동 교차 검증 (샘플링 반복 횟수 `n_iter` 매개변수로 지정)
<br/><br/>

### Hyper Parameter Tuning 순서
1. 서치 방법 객체 정의
2. 매개변수 딕셔너리 서치 객체 전달
3. 서치방법 학습 (`fit`)
4. `best_estimator_`: 최적의 매개변수로 이루어진 모델을 훈련세트로 훈련까지 마친 최적의 모델 저장
5. `best_params_`: 최적의 매개변수 값 저장
6. `cv_results_`: fit time, score time, test score 값을 딕셔너리로 반환
7. `np.argmax()`: `cv_results_` 속성 사용시 인덱스 반환
8. `np.max()`: `cv_results_` 속성 사용시 최고 값 반환 (e.g., `np.max(gs.cv_results_['mean_test_score'])`)

### 확인문제
- 위에 있는 매개변수 딕셔너리 그대로 이용하는데 splitter 매개변수를 'random'으로 지정했을 때 최적의 매개변수값과 최적의 모델에 테스트 세트 적용했을 때의 정확도 점수를 확인해보자

In [23]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
    # RandomizedSearchCV 객제 정의: 결정 트리 모델(랜덤 분할), 매개 변수 딕셔너리, 100회 반복 
gs.fit(train_data, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fc2f8e1e4d0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fc32b3ef400>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fc2f8e1fc10>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fc32b3ef2e0>},
                   random_state=42)

In [24]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_data, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
